<a href="https://colab.research.google.com/gist/gretahartm/632c642fee3d61e041f06aabd8c25433/6notebooksb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train U-Net Tracker Model on Google Colab

This notebook provides a step-by-step guide to train a U-Net-based model (`unet-tracker`) for face tracking using Google Colab. With a training set of approximately 200-400 images, training for 200 epochs yielded reasonable results.

---

## Prerequisites

### 1. Enable GPU Access
To leverage GPU acceleration for faster training:
1. Go to the menu: **Runtime > Change runtime type**.
2. Set the **Hardware Accelerator** to **GPU**.

### 2. Install `unet-tracker`


*   Prepare the dataset
*   Configure the training parameters
*   Train and evaluate your U-Net tracker model






In [1]:
!pip install albumentations==1.3.0
!git clone https://github.com/kevin-allen/unetTracker
!pip install -r unetTracker/requirements.txt
!pip install -e unetTracker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20
Cloning into 'unetTracker'...
remote: Enumerating objects: 1022, done.
remote: Counting objects: 100% (401/401), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 1022 (delta 233), reused 353 (delta 204), pack-reused 621 (from 1)
Receiving objects: 100% (1022/1022), 127.56 MiB | 14.62 MiB/s, done.
Resolving deltas: 100% (623/623), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.7/515.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.

In [2]:
from google.colab import drive
import os

drive.mount('/content/drive')

fn = "/content/drive/My Drive/ssfolder"
if os.path.exists(fn):
  print("We can access the ssfolder directory.")
else:
  raise IOError("Problem accessing the ssfolder directory.")

Mounted at /content/drive
We can access the ssfolder directory.


In [3]:
# this will run the code in the setup_project.py and create a variable called `project`
fn = "/content/drive/My Drive/ssfolder/setup_project.py"
if os.path.exists(fn):
  print("We can access the file.")
else:
  raise IOError("Problem accessing the file.")

%run "/content/drive/My Drive/ssfolder/setup_project.py"

We can access the file.
Project directory: /content/drive/My Drive/ssfolder/trackingProjects/simonebiles_tracker
Getting configuration from config file. Values from config file will be used.
Loading /content/drive/My Drive/ssfolder/trackingProjects/simonebiles_tracker/config.yalm
{'augmentation_HorizontalFlipProb': 0.5, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_extension': '.png', 'image_size': [270, 480], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'simonebiles_tracker', 'normalization_values': {'means': [0.1952759027481079, 0.15899455547332764, 0.13951754570007324], 'stds': [0.16387943923473358, 0.16304421424865723, 0.16733983159065247]}, 'object_colors': [(240.0, 248.0, 255.0), (250.0, 235.0, 215.0), (0.0, 255.0, 255.0), (127.0, 255.0, 212.0), (240.0, 255.0, 255.0), (245.0, 245.0, 220.0), (255.0, 228.0, 196.0), (0.0, 0.0, 0.0), (255.0, 235.0, 205.0), (0.0, 0.0, 255.0)], 'objects': ['head', 'right

In [4]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch import optim
import torch.nn as nn
from datetime import datetime
import albumentations as A
import cv2
import os
import pickle

from unetTracker.trackingProject import TrackingProject
from unetTracker.dataset import UNetDataset
from unetTracker.unet import Unet
from unetTracker.coordinatesFromSegmentationMask import CoordinatesFromSegmentationMask
from unetTracker.utils import check_accuracy

In [5]:
torch.cuda.is_available(),torch.cuda.get_device_name(0)

(True, 'Tesla T4')

#Hyperparamaters


*   when first time training model --> LOAD_MODEL = False



In [6]:
LEARNING_RATE=1e-4
DEVICE = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
BATCH_SIZE=4
NUM_EPOCHS = 100
NUM_WORKERS = 2
OUTPUT_CHANNELS = len(project.object_list)
IMAGE_HEIGHT = project.image_size[0]
IMAGE_WIDTH = project.image_size[1]
PIN_MEMORY = True
LOAD_MODEL = True # If this is the first time you train the model, set LOAD_MODEL to False
TRAIN_IMAGE_DIR = os.path.join(project.dataset_dir,"train_images")
TRAIN_MASK_DIR =  os.path.join(project.dataset_dir,"train_masks")
TRAIN_COORDINATE_DIR = os.path.join(project.dataset_dir,"train_coordinates")
VAL_IMAGE_DIR = os.path.join(project.dataset_dir,"val_images")
VAL_MASK_DIR =  os.path.join(project.dataset_dir,"val_masks")
VAL_COORDINATE_DIR = os.path.join(project.dataset_dir,"val_coordinates")

#Model, loss, and optimizer

In [7]:
model = Unet(in_channels=3, out_channels=OUTPUT_CHANNELS).to(DEVICE)
if LOAD_MODEL:
    project.load_model(model)

# set the model in train mode
model.train()

loss_fn = nn.BCEWithLogitsLoss() # not doing sigmoid on the output of the model, so use this, if we had more classes (objects) we would use change out_chan and cross_entropy_loss as loss_fn
optimizer= optim.Adam(model.parameters(),lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()

/content/drive/My Drive/ssfolder/unetTracker/trackingProject.py:127: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(self.model_fn))
<ipython-

#Data augmentation and normalization

In [8]:
fileName = os.path.join(project.augmentation_dir,"trainTransform")
print("Loading trainTransform from", fileName)
trainTransform=pickle.load(open(fileName,"rb" ))

fileName = os.path.join(project.augmentation_dir,"valTransform")
print("Loading valTransform from", fileName)
valTransform=pickle.load(open(fileName, "rb" ))

Loading trainTransform from /content/drive/My Drive/ssfolder/trackingProjects/simonebiles_tracker/augmentation/trainTransform
Loading valTransform from /content/drive/My Drive/ssfolder/trackingProjects/simonebiles_tracker/augmentation/valTransform


#Datasets and DataLoaders

In [9]:
trainDataset = UNetDataset(TRAIN_IMAGE_DIR, TRAIN_MASK_DIR,TRAIN_COORDINATE_DIR, transform=trainTransform)
valDataset = UNetDataset(VAL_IMAGE_DIR, VAL_MASK_DIR,VAL_COORDINATE_DIR, transform=valTransform)
trainLoader = DataLoader(trainDataset,shuffle=True,batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,pin_memory=PIN_MEMORY)
valLoader = DataLoader(valDataset,shuffle=False,batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,pin_memory = PIN_MEMORY)

In [10]:
imgs, masks, _ = next(iter(trainLoader))
imgs.shape, masks.shape

(torch.Size([4, 3, 270, 480]), torch.Size([4, 10, 270, 480]))

#Save and load checkpoint

In [11]:
def save_checkpoint(state, filename = "my_checkpoint.pth.tar"):
    #print("Saving checkpoint")
    torch.save(state,filename)

#Train the Model


## Purpose
- Train the model to predict masks for images in the dataset.
- Evaluate model performance on the validation set every few epochs.

## Task
- **Input**: Each image in the dataset with its corresponding labeled masks.
- **Output**: Predicted masks matching the labeled masks created during annotation.
  - One mask per body part per image.
  - Masks are the same size as the images.
  - Mask structure:
    - Most pixels set to `0`.
    - A circle centered on the body part is set to `1`.

## Model Objective
- Learn to output values for each mask that closely resemble the labeled masks.
- Use a **loss function** to measure the similarity between the labeled masks and model outputs.

## Training Process
1. Start training the model.
2. Evaluate the performance periodically on the validation set.
3. Observe model progress:
   - Loss value should decrease over time.
   - Typically, it takes **40-60 epochs** for the model to start identifying the labeled positive pixels as positive in the output masks.


In [12]:
def train_fn(loader,model,optimizer,loss_fn,scaler,epoch,total_epochs):
    """
    One epoch of training
    """
    loop = tqdm(loader)
    for batch_idx, (data,targets,_) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions,targets)


        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix_str("loss: {:.7f}, epoch: {:d}/{:d}".format(loss.item(),epoch,total_epochs))

In [13]:
startTime = datetime.now()
print("Starting time:",startTime)
for epoch in range(NUM_EPOCHS):

    train_fn(trainLoader,model,optimizer,loss_fn,scaler,epoch,NUM_EPOCHS)

    if epoch % 5 == 0 :
        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict()}
        save_checkpoint(checkpoint,filename=os.path.join(project.models_dir,"my_checkpoint.pth.tar"))

        # check accuracy
        print("Performance on validation set")
        check_accuracy(model,valLoader,DEVICE)

endTime=datetime.now()
print("End time:",endTime)
print("{} epochs, duration:".format(NUM_EPOCHS), endTime-startTime)

Starting time: 2025-01-06 09:57:38.726883


  0%|          | 0/35 [00:00<?, ?it/s]<ipython-input-12-4c8d78ac898b>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 35/35 [00:28<00:00,  1.23it/s, loss: 0.0004380, epoch: 0/100]


Performance on validation set
Number of positive pixels predicted: 6146.0
Number of positive pixels in masks: 8190.0
Percentage of positive pixels predicted: 0.025
Percentage of positive pixels in masks: 0.033
Accuracy: 99.962
Dice score: 0.353
Mask pixels detected (True positives): 30.989%
False positives: 58.705%
Mean distance: 5.599689683234941


100%|██████████| 35/35 [00:09<00:00,  3.77it/s, loss: 0.0001904, epoch: 5/100]


Performance on validation set
Number of positive pixels predicted: 5386.0
Number of positive pixels in masks: 8190.0
Percentage of positive pixels predicted: 0.022
Percentage of positive pixels in masks: 0.033
Accuracy: 99.965
Dice score: 0.361
Mask pixels detected (True positives): 29.829%
False positives: 54.642%
Mean distance: 7.5692338672005475


...

Performance on validation set
Number of positive pixels predicted: 5522.0
Number of positive pixels in masks: 8190.0
Percentage of positive pixels predicted: 0.022
Percentage of positive pixels in masks: 0.033
Accuracy: 99.963
Dice score: 0.326
Mask pixels detected (True positives): 27.363%
False positives: 59.417%
Mean distance: 7.413131688114566


100%|██████████| 35/35 [00:08<00:00,  3.94it/s, loss: 0.0004215, epoch: 95/100]


Performance on validation set
Number of positive pixels predicted: 5605.0
Number of positive pixels in masks: 8190.0
Percentage of positive pixels predicted: 0.023
Percentage of positive pixels in masks: 0.033
Accuracy: 99.962
Dice score: 0.327
Mask pixels detected (True positives): 27.692%
False positives: 59.536%
Mean distance: 8.370535901466033


100%|██████████| 35/35 [00:09<00:00,  3.75it/s, loss: 0.0002153, epoch: 99/100]

End time: 2025-01-06 10:14:50.849933
100 epochs, duration: 0:17:12.123050


In [14]:
project.save_model(model)

saving model state dict to /content/drive/My Drive/ssfolder/trackingProjects/simonebiles_tracker/models/UNet.pt
2025-01-06 10:14:54.446354
